# Complete BYOK Example App (Optimized)

> An optimized FastHTML application demonstrating the BYOK system with improved UI design using semantic colors and Refactoring UI principles

In [1]:
#| default_exp examples.complete_app_optimized

## Setup and Imports

In [2]:
import os
from datetime import timedelta
from fasthtml.common import *
from fasthtml.jupyter import *

# BYOK Core imports
from cjm_fasthtml_byok.core.storage import BYOKManager
from cjm_fasthtml_byok.core.types import BYOKConfig, StorageBackend
from cjm_fasthtml_byok.middleware.beforeware import create_byok_beforeware, require_api_key
from cjm_fasthtml_byok.utils.helpers import get_key_summary, import_from_env, format_provider_name

# BYOK Component imports
from cjm_fasthtml_byok.components.forms import (
    KeyInputForm, 
    MultiProviderKeyForm,
    KeyManagementCard,
    KeyManagerDashboard,
    InlineKeyInput
)
from cjm_fasthtml_byok.components.alerts import (
    Alert,
    SecurityAlert,
    KeyStatusNotification,
    ValidationMessage,
    ToastContainer
)

# Optimized UI library imports
from cjm_fasthtml_daisyui.core.resources import get_daisyui_headers
from cjm_fasthtml_daisyui.core.testing import create_theme_selector

# DaisyUI Components
from cjm_fasthtml_daisyui.components.navigation.navbar import navbar, navbar_start, navbar_center, navbar_end
from cjm_fasthtml_daisyui.components.actions.button import btn, btn_colors, btn_sizes, btn_modifiers, btn_styles
from cjm_fasthtml_daisyui.components.data_display.card import card, card_body, card_title, card_actions
from cjm_fasthtml_daisyui.components.data_display.badge import badge, badge_colors, badge_sizes
from cjm_fasthtml_daisyui.components.feedback.alert import alert, alert_colors
from cjm_fasthtml_daisyui.components.layout.hero import hero, hero_content
from cjm_fasthtml_daisyui.components.layout.divider import divider

# Semantic Colors - CRITICAL for theme support
from cjm_fasthtml_daisyui.utilities.semantic_colors import (
    bg_dui, text_dui, border_dui, shadow_dui
)

# Tailwind Utilities
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import (
    flex_display, flex_center, flex_between, gap, grid_cols, items, justify, responsive_grid, flex
)
from cjm_fasthtml_tailwind.utilities.spacing import p, m, space
from cjm_fasthtml_tailwind.utilities.sizing import w, h, max_w, min_h, container
from cjm_fasthtml_tailwind.utilities.typography import font_size, font_weight, leading, font_family
from cjm_fasthtml_tailwind.utilities.effects import shadow, ring, ring_color
from cjm_fasthtml_tailwind.utilities.borders import rounded, border
from cjm_fasthtml_tailwind.utilities.transitions_and_animation import hover_effect, smooth_transition
from cjm_fasthtml_tailwind.core.base import combine_classes

## Create the FastHTML Application

In [3]:
# Configuration
SECRET_KEY = os.environ.get("SECRET_KEY", "dev-secret-key-change-in-production")
DATABASE_URL = "sqlite:///byok_demo.db"  # Use SQLite file for demo

# Configure BYOK
byok_config = BYOKConfig(
    storage_backend=StorageBackend.HYBRID,  # Use both session and database
    default_ttl=timedelta(hours=24),
    auto_cleanup=True,
    require_https=False  # Disabled for demo
)

# Initialize BYOK manager with SQLAlchemy
byok = BYOKManager(
    secret_key=SECRET_KEY,
    db_url=DATABASE_URL,  # Pass database URL instead of db object
    config=byok_config
)

# Setup BYOK beforeware
def get_user_id(req):
    """Get user ID from session (mock for demo)"""
    # Access session from req.session, not req.scope['session']
    if hasattr(req, 'session'):
        return req.session.get("user_id", "demo-user")
    return "demo-user"

# Create the beforeware handler
byok_beforeware = create_byok_beforeware(byok)

# Create the FastHTML app with session support and beforeware
from starlette.middleware.sessions import SessionMiddleware

app, rt = fast_app(
    title="Dashboard - BYOK Demo",
    pico=False,
    hdrs=get_daisyui_headers(include_themes=True),
    secret_key=SECRET_KEY,
    sess_cls=SessionMiddleware,  # Use proper SessionMiddleware
    before=byok_beforeware,  # Add the beforeware here
    live=False
)

## Define Optimized Layout Components

In [4]:
def AppNavbar(req, active_page="dashboard"):
    """Create an optimized application navbar with better visual hierarchy"""
    
    # Define navigation items with active state
    nav_items = [
        ("Dashboard", "/", "dashboard"),
        ("Add Key", "/add", "add"),
        ("Settings", "/settings", "settings")
    ]
    
    return Div(
        Div(
            # Logo with better styling
            A(
                Span("🔑", cls=combine_classes(font_size._2xl, m.r(2))),
                Span("BYOK Demo", cls=combine_classes(
                    font_weight.bold,
                    font_size.xl,
                    text_dui.primary  # Use semantic color
                )),
                href="/",
                cls=combine_classes(
                    btn,
                    btn_styles.ghost,
                    hover_effect()  # Add smooth transition
                )
            ),
            cls=str(navbar_start)
        ),
        Div(
            *[
                A(
                    name,
                    href=href,
                    cls=combine_classes(
                        btn,
                        btn_styles.ghost if page != active_page else btn_colors.primary,
                        btn_sizes.sm,  # Smaller buttons for cleaner look
                        smooth_transition()
                    )
                )
                for name, href, page in nav_items
            ],
            create_theme_selector(),
            cls=combine_classes(navbar_end, gap(2))  # Add gap between buttons
        ),
        cls=combine_classes(
            navbar,
            bg_dui.base_200,  # Slightly darker background
            shadow.md,  # Add subtle shadow for depth
            p.y(2)  # More vertical padding
        )
    )

def PageLayout(req, title: str, *content, full_width=False, active_page="dashboard"):
    """Optimized page layout with better spacing and visual hierarchy
    
    Args:
        req: Request object
        title: Page title
        *content: Page content
        full_width: If True, don't wrap content in a container
        active_page: Current active page for navbar highlighting
    """
    if full_width:
        # Don't add container wrapper for components that manage their own layout
        page_content = Div(
            *content,
            cls=combine_classes(
                min_h.screen,  # Full height
                bg_dui.base_100  # Proper background
            )
        )
    else:
        # Standard container with better spacing
        page_content = Div(
            *content,
            cls=combine_classes(
                container,
                m.x.auto,
                p(6),  # More generous padding
                max_w.screen_xl,
                min_h.screen,
                bg_dui.base_100
            )
        )
    
    return Div(
        Head(Title(title)),
        AppNavbar(req, active_page=active_page),
        page_content,
        ToastContainer(position="top", align="end")
    )

def PageHeader(title: str, subtitle: str = None):
    """Consistent page header with visual hierarchy"""
    return Div(
        H1(
            title,
            cls=combine_classes(
                font_size._3xl,
                font_weight.bold,
                text_dui.base_content,
                m.b(2)
            )
        ),
        P(
            subtitle,
            cls=combine_classes(
                font_size.lg,
                text_dui.base_content.opacity(70),  # Faded for hierarchy
                m.b(8)
            )
        ) if subtitle else None
    )

def StyledAlert(message: str, kind: str = "info", dismissible: bool = True):
    """Enhanced alert with semantic colors and better styling"""
    # Map kind to semantic colors
    color_map = {
        "success": (alert_colors.success, "✓"),
        "error": (alert_colors.error, "✕"),
        "warning": (alert_colors.warning, "⚠"),
        "info": (alert_colors.info, "ℹ")
    }
    
    alert_color, icon = color_map.get(kind, (alert_colors.info, "ℹ"))
    
    return Div(
        Span(icon, cls=combine_classes(font_size.xl, m.r(3))),
        Span(message),
        Button(
            "✕",
            cls=combine_classes(btn, btn_styles.ghost, btn_sizes.sm, m.l.auto),
            onclick="this.parentElement.remove()"
        ) if dismissible else None,
        role="alert",
        cls=combine_classes(
            alert,
            alert_color,
            shadow.sm,  # Subtle shadow for depth
            m.b(4)
        )
    )

## Optimized Route Handlers

In [5]:
@rt("/")
def index(req, sess):
    """Optimized main dashboard page with better visual design"""
    # Set user ID in session
    sess["user_id"] = "demo-user"
    
    # Get stored keys summary
    providers = ["openai", "anthropic", "google", "groq", "fireworks", "xai"]
    user_id = get_user_id(req)
    
    # Check for any notifications
    notifications = []
    if msg := req.query_params.get("msg"):
        msg_type = req.query_params.get("type", "info")
        notifications.append(StyledAlert(msg, kind=msg_type, dismissible=True))
    
    return PageLayout(
        req,
        "Dashboard - BYOK Demo",
        # Hero section for better first impression
        Div(
            Div(
                PageHeader(
                    "API Key Management Dashboard",
                    "Securely manage your API keys across multiple providers"
                ),
                Div(*notifications) if notifications else None,
                cls=combine_classes(
                    hero_content,
                    p.y(8),  # Generous vertical padding
                    text_dui.base_content
                )
            ),
            cls=combine_classes(hero, bg_dui.base_200, m.b(8))
        ),
        # Dashboard with proper container
        Div(
            KeyManagerDashboard(
                req,
                providers=providers,
                byok_manager=byok,
                user_id=user_id,
                base_url="/api/keys"
            ),
            cls=combine_classes(container, m.x.auto, p.x(4))
        ),
        full_width=True,
        active_page="dashboard"
    )

In [6]:
@rt("/add")
def add_key_page(req, sess):
    """Optimized page to add a new API key with better form design"""
    providers = ["openai", "anthropic", "google", "groq", "fireworks", "xai"]
    
    return PageLayout(
        req,
        "Add API Key - BYOK Demo",
        PageHeader(
            "Add New API Key",
            "Store a new API key securely for use across your application"
        ),
        # Card wrapper for the form
        Div(
            Div(
                MultiProviderKeyForm(
                    providers=providers,
                    action="/api/keys/add"
                ),
                cls=combine_classes(
                    card_body,
                    p(8),  # Generous padding
                    space.y(6)  # Spacing between form elements
                )
            ),
            cls=combine_classes(
                card,
                bg_dui.base_200,  # Slightly different background
                shadow.lg,  # Stronger shadow for emphasis
                max_w.lg,
                m.x.auto
            )
        ),
        active_page="add"
    )

In [7]:
@rt("/settings")
def settings_page(req, sess):
    """Optimized settings page with better visual organization"""
    user_id = get_user_id(req)
    summary = get_key_summary(byok, req, user_id)
    
    return PageLayout(
        req,
        "Settings - BYOK Demo",
        PageHeader(
            "Settings",
            "Manage your security settings and stored keys"
        ),
        
        # Security status card
        Div(
            Div(
                H3(
                    "Security Status",
                    cls=combine_classes(
                        card_title,
                        font_size.xl,
                        text_dui.base_content
                    )
                ),
                Div(cls=str(divider)),  # Visual separator
                SecurityAlert(
                    "HTTPS is not enabled. API keys may be transmitted insecurely.",
                    severity="medium" if req.url.scheme == "http" else "low"
                ) if req.url.scheme == "http" else StyledAlert(
                    "Connection is secure (HTTPS)",
                    kind="success"
                ),
                cls=str(card_body)
            ),
            cls=combine_classes(card, bg_dui.base_200, shadow.md, m.b(6))
        ),
        
        # Key summary card
        Div(
            Div(
                H3(
                    "Stored Keys Summary",
                    cls=combine_classes(
                        card_title,
                        font_size.xl,
                        text_dui.base_content
                    )
                ),
                Div(cls=str(divider)),
                # Stats display
                Div(
                    Div(
                        Span(
                            str(summary['count']),
                            cls=combine_classes(
                                font_size._3xl,
                                font_weight.bold,
                                text_dui.primary
                            )
                        ),
                        Span(
                            " Total Keys Stored",
                            cls=combine_classes(
                                font_size.lg,
                                text_dui.base_content.opacity(70)
                            )
                        ),
                        cls=combine_classes(m.b(4))
                    ),
                    # Key list with better styling
                    Div(
                        *[
                            Div(
                                Span(
                                    key['display_name'],
                                    cls=combine_classes(
                                        badge,
                                        badge_colors.primary,
                                        badge_sizes.lg,
                                        m.r(2)
                                    )
                                ),
                                Span(
                                    key['masked_key'],
                                    cls=combine_classes(
                                        font_family.mono,
                                        text_dui.base_content.opacity(60)
                                    )
                                ),
                                Span(
                                    f" • {key['created']}",
                                    cls=combine_classes(
                                        font_size.sm,
                                        text_dui.base_content.opacity(50)
                                    )
                                ),
                                cls=combine_classes(
                                    p(3),
                                    bg_dui.base_100,
                                    rounded.lg,
                                    m.b(2),
                                    hover_effect()
                                )
                            )
                            for key in summary['keys']
                        ],
                        cls=str(space.y(2))
                    ) if summary['keys'] else P(
                        "No keys stored",
                        cls=combine_classes(
                            text_dui.base_content.opacity(50),
                            font_style.italic
                        )
                    )
                ),
                cls=str(card_body)
            ),
            cls=combine_classes(card, bg_dui.base_200, shadow.md, m.b(6))
        ),
        
        # Actions card with danger zone styling
        Div(
            Div(
                H3(
                    "Danger Zone",
                    cls=combine_classes(
                        card_title,
                        font_size.xl,
                        text_dui.error  # Red for danger
                    )
                ),
                Div(cls=combine_classes(divider, border_dui.error.opacity(30))),
                P(
                    "This action cannot be undone. All stored API keys will be permanently deleted.",
                    cls=combine_classes(
                        text_dui.base_content.opacity(70),
                        m.b(4)
                    )
                ),
                Form(
                    Button(
                        "Clear All Keys",
                        type="submit",
                        cls=combine_classes(
                            btn,
                            btn_colors.error,
                            btn_modifiers.wide,  # Full width for emphasis
                            shadow.md
                        ),
                        onclick="return confirm('⚠️ Are you absolutely sure? This will delete ALL stored API keys.');"
                    ),
                    method="post",
                    action="/api/keys/clear-all"
                ),
                cls=str(card_body)
            ),
            cls=combine_classes(
                card,
                bg_dui.base_200,
                border(2),
                border_dui.error.opacity(50),  # Red border for danger
                shadow.md
            )
        ),
        active_page="settings"
    )

## API Endpoints (Same as original but with better redirects)

In [8]:
@rt("/api/keys/add", methods=["POST"])
def add_key(req, sess, provider: str, api_key: str):
    """Add a new API key"""
    user_id = get_user_id(req)
    
    try:
        # Store the key
        byok.set_key(
            req,
            provider=provider,
            api_key=api_key,
            user_id=user_id,
            ttl=timedelta(days=30)
        )
        
        return RedirectResponse(
            url=f"/?msg=✓+{format_provider_name(provider)}+key+added+successfully&type=success",
            status_code=303
        )
    except Exception as e:
        return RedirectResponse(
            url=f"/?msg=Failed+to+add+key:+{str(e)}&type=error",
            status_code=303
        )

@rt("/api/keys/{provider}", methods=["POST"])
def update_key(req, sess, provider: str, api_key: str):
    """Update an API key for a specific provider"""
    user_id = get_user_id(req)
    
    try:
        byok.set_key(
            req,
            provider=provider,
            api_key=api_key,
            user_id=user_id,
            ttl=timedelta(days=30)
        )
        
        return RedirectResponse(
            url=f"/?msg=✓+{format_provider_name(provider)}+key+updated&type=success",
            status_code=303
        )
    except Exception as e:
        return RedirectResponse(
            url=f"/?msg=Failed+to+update+key:+{str(e)}&type=error",
            status_code=303
        )

@rt("/api/keys/{provider}/delete", methods=["POST"])
def delete_key(req, sess, provider: str):
    """Delete an API key"""
    user_id = get_user_id(req)
    
    try:
        byok.delete_key(req, provider, user_id)
        
        return RedirectResponse(
            url=f"/?msg={format_provider_name(provider)}+key+deleted&type=info",
            status_code=303
        )
    except Exception as e:
        return RedirectResponse(
            url=f"/?msg=Failed+to+delete+key:+{str(e)}&type=error",
            status_code=303
        )

@rt("/api/keys/clear-all", methods=["POST"])
def clear_all_keys(req, sess):
    """Clear all stored API keys"""
    user_id = get_user_id(req)
    
    try:
        byok.clear_keys(req, user_id)
        
        return RedirectResponse(
            url="/?msg=⚠️+All+keys+have+been+cleared&type=warning",
            status_code=303
        )
    except Exception as e:
        return RedirectResponse(
            url=f"/?msg=Failed+to+clear+keys:+{str(e)}&type=error",
            status_code=303
        )

## Optimized Protected Routes Example

In [9]:
@rt("/demo/openai")
@require_api_key("openai", user_id_func=get_user_id)
def openai_demo(req, sess):
    """Optimized demo route with better visual design"""
    user_id = get_user_id(req)
    # Get the API key using the global byok manager
    api_key = byok.get_key(req, "openai", user_id)
    
    return PageLayout(
        req,
        "OpenAI Demo - BYOK",
        PageHeader(
            "OpenAI Integration",
            "This page demonstrates protected route access with API keys"
        ),
        
        # Success card
        Div(
            Div(
                # Success badge
                Div(
                    Span(
                        "✓ Authenticated",
                        cls=combine_classes(
                            badge,
                            badge_colors.success,
                            badge_sizes.lg,
                            m.b(4)
                        )
                    )
                ),
                
                # Key preview with better styling
                Div(
                    Span(
                        "API Key:",
                        cls=combine_classes(
                            font_weight.semibold,
                            text_dui.base_content,
                            m.r(2)
                        )
                    ),
                    Code(
                        f"{api_key[:10]}...{api_key[-4:]}",
                        cls=combine_classes(
                            bg_dui.base_300,
                            text_dui.primary,
                            p.x(3),
                            p.y(1),
                            rounded.md,
                            font_family.mono
                        )
                    ),
                    cls=combine_classes(m.b(4))
                ),
                
                Div(cls=str(divider)),
                
                # Information
                P(
                    "This page requires an OpenAI API key to access.",
                    cls=combine_classes(
                        text_dui.base_content,
                        m.b(2)
                    )
                ),
                P(
                    "In a real application, you would use this key to make API calls to OpenAI services like GPT-4, DALL-E, or Whisper.",
                    cls=combine_classes(
                        text_dui.base_content.opacity(70),
                        m.b(4)
                    )
                ),
                
                # Example code snippet
                Div(
                    H4(
                        "Example Usage:",
                        cls=combine_classes(
                            font_weight.semibold,
                            text_dui.base_content,
                            m.b(2)
                        )
                    ),
                    Pre(
                        Code(
                            """import openai

# Your API key is securely retrieved
openai.api_key = api_key

# Make API calls
response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[{"role": "user", "content": "Hello!"}]
)""",
                            cls=combine_classes(
                                bg_dui.base_300,
                                text_dui.base_content,
                                p(4),
                                rounded.lg,
                                font_family.mono,
                                font_size.sm
                            )
                        )
                    )
                ),
                cls=str(card_body)
            ),
            cls=combine_classes(
                card,
                bg_dui.base_200,
                shadow.lg,
                max_w._2xl,
                m.x.auto
            )
        )
    )

## Run the Application

In [12]:
#| eval: false
# For Jupyter notebook display
from fasthtml.jupyter import *
from cjm_fasthtml_daisyui.core.testing import start_test_server

# Start the server
server = start_test_server(app, port=5001)

# Display in Jupyter
display(HTMX(port=server.port))

print("🚀 Optimized BYOK Demo App is running!")
print(f"📍 Visit http://localhost:{server.port} to see the app")
print("\n✨ Improvements in this version:")
print("  • Semantic colors for theme support")
print("  • Better visual hierarchy with proper typography")
print("  • Enhanced spacing following Refactoring UI principles")
print("  • Improved cards with shadows and backgrounds")
print("  • Active navigation state highlighting")
print("  • Better form and alert styling")
print("  • Consistent use of DaisyUI components")
print("  • Smooth transitions and hover effects")
print("\nPress Stop button in Jupyter to stop the server")

🚀 Optimized BYOK Demo App is running!
📍 Visit http://localhost:5001 to see the app

✨ Improvements in this version:
  • Semantic colors for theme support
  • Better visual hierarchy with proper typography
  • Enhanced spacing following Refactoring UI principles
  • Improved cards with shadows and backgrounds
  • Active navigation state highlighting
  • Better form and alert styling
  • Consistent use of DaisyUI components
  • Smooth transitions and hover effects

Press Stop button in Jupyter to stop the server


In [13]:
#| eval: false
# Stop the server when done
server.stop()